# Rechenaufgaben zu 4.2 (Passives Reinforcement Learning)

---

## Rechenaufgabe

In einer vollautomatisierten Packstation soll ein Transportroboter eigenständig Pakete von der Annahmestelle in die Regale transportieren. 

Der ideale Ablauf hierfür ist, dass er das Paket aufnimmt (load), sich zu den Regalen bewegt (move), dort das Paket ablegt (unload) und zurückkehrt.

Der Zielzustand ist erreicht, wenn sich der Roboter an der Annahmestelle befindet und kein weiteres Paket vorliegt. Die Menge der Zustände ergibt sich folglich aus dem Aufenthaltsort des Roboters (a Annahmestelle oder r Regale) und dem Status des zu bearbeitenden Paketes (v vorgelegt, u unterwegs, w weggebracht).

$$  
S_\text{tr} = \{s_0=s_a^{v},s_a^{u},s_r^{v},s_r^{u},s_r^{w},s^t=s_a^{w}\}
$$

Die Menge der Aktionen des Transporters ist

$$
A_\text{tr}=\{\textit{move},\textit{load},\textit{unload}\}   
$$

Weiterhin sei folgende Strategie $\pi_\text{tr}$ bereits vorgegeben:
$$
\pi_\text{tr}(s_a^{v})  = \textit{load}\\    \pi_\text{tr}(s_r^{v})  = \pi_\text{tr}(s_r^{w}) = \pi_\text{tr}(s_a^{u}) = \textit{move}\\    \pi_\text{tr}(s_r^{u})  = \textit{unload}
$$

Mit dieser werden die folgenden fünf Probeläufe absolviert.

---

$$
O_1  = ((s_a^{v},\textit{load},s_a^{u},3),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{w},8),(s_r^{w},\textit{move},s_a^{w},0))
$$

---

$$
O_2  = ((s_a^{v},\textit{load},s_a^{v},-1),(s_a^{v},\textit{load},s_a^{u},3),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{v},8),(s_r^{v},\textit{move},s_a^{v},0),\\
(s_a^{v},\textit{load},s_a^{u},3),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{w},8),(s_r^{w},\textit{move},s_a^{w},0))
$$

---

$$
O_3  = ((s_a^{v},\textit{load},s_a^{u},3),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{w},8),(s_r^{w},\textit{move},s_a^{w},0))
$$

---

$$
O_4  = ((s_a^{v},\textit{load},s_a^{u},3),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{v},8),(s_r^{v},\textit{move},s_a^{v},0),(s_a^{v},\textit{load},s_a^{u},3),\\
(s_a^{u},\textit{move},s_a^{u},-1),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{w},8),(s_r^{w},\textit{move},s_a^{w},0))
$$

---

$$
O_5  = ((s_a^{v},\textit{load},s_a^{u},3),(s_a^{u},\textit{move},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{u},-1),(s_r^{u},\textit{unload},s_r^{w},8),(s_r^{w},\textit{move},s_a^{w},0))
$$

---

**Aufgaben:**

1. Bestimmen Sie für jeden Zustand $s$ aus $S\setminus{S_t}$ die Multimenge $\mathcal{O}_s$ bezüglich $\mathcal{O}=(O_1,O_2,O_3,O_4,O_5)$, sowie $U^\gamma_\mathcal{O}(s\mid\pi_{tr})$, mit $\gamma=0.9$.
2. Berechnen Sie die für die adaptive dynamische Programmierung benötigten Zustandsübergangswahrscheinlichkeiten $\tilde{P}_\mathcal{O}(s,\pi_{tr}(s),s')$ für die Zustände $s_r^{u}$ und $s_{a}^{v}$
3. Zur Initialisierung des TD-Algorithmus seien folgende Zustandsnutzen gegeben:

    $$
    u^\gamma(s^{v}_a\;|\;\pi_\text{tr}) = u^\gamma(s^{u}_a\;|\; \pi_\text{tr}) = u^\gamma(s^{u}_r\;|\;\pi_\text{tr}) = u^\gamma(s^{v}_r\;|\;\pi_\text{tr})= u^\gamma(s^{w}_r\;|\; \pi_\text{tr}) = 0
    $$

    Aktualisieren Sie die Zustandsnutzen nach den Beobachtungen aus dem dritten Probedurchlauf $O_3$ entsprechend der Update-Regel des Kurstextes.\
    Dabei sei $\gamma=0.9$ und $\alpha=\frac{1}{n}$, wobei $n$ die Anzahl der bisher gemachten Beobachtungen ist (beginnend bei $n=1$).


In [86]:
# Das machen wir am besten programmatisch.

state_space = ["s_a^v", "s_a^u", "s_r^v", "s_r^u", "s_r^w", "s_a^w"]

strategy = {
    "s_a^v": "load",
    "s_r^v": "move",
    "s_r^w": "move",
    "s_a^u": "move",
    "s_r^u": "unload",
}

O_1 = [
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^w", 8),
    ("s_r^w", "move", "s_a^w", 0),
]

O_2 = [
    ("s_a^v", "load", "s_a^v", -1),
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^v", 8),
    ("s_r^v", "move", "s_a^v", 0),
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^w", 8),
    ("s_r^w", "move", "s_a^w", 0),
]

O_3 = [
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^w", 8),
    ("s_r^w", "move", "s_a^w", 0),
]

O_4 = [
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^v", 8),
    ("s_r^v", "move", "s_a^v", 0),
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_a^u", -1),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^w", 8),
    ("s_r^w", "move", "s_a^w", 0),
]

O_5 = [
    ("s_a^v", "load", "s_a^u", 3),
    ("s_a^u", "move", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^u", -1),
    ("s_r^u", "unload", "s_r^w", 8),
    ("s_r^w", "move", "s_a^w", 0),
]

list_of_trials = [O_1, O_2, O_3, O_4, O_5]
all_observations = [observation for trial in list_of_trials for observation in trial]

In [89]:
trials_for_state = {state: [] for state in state_space}
for state in state_space:
    for trial in list_of_trials:
        sub_trial_observations_for_state = []
        for observation in trial:
            # Add observation to all known sub-trials
            for trial_idx in range(len(sub_trial_observations_for_state)):
                sub_trial_observations_for_state[trial_idx] += [observation]

            # Create a new trial
            if observation[0] == state:
                sub_trial_observations_for_state += [[observation]]

        trials_for_state[state] += sub_trial_observations_for_state

print("Aufgabe 1 - bestimmung von O_s")
for state in state_space:
    print(f"{state}: {trials_for_state[state]}")


gamma = 0.9


def berechne_nutzen_for_state(trials):
    if len(trials) == 0:
        return 0

    total_nutzen = 0
    for trial in trials:
        for i, observation in enumerate(trial):
            total_nutzen += observation[-1] * (gamma**i)

    return total_nutzen / len(trials)


print(
    "------------------------------------------------------------------------------------------------------------------"
)

print("Aufgabe 1 - Berechnung der Zustandsnutzen")
for state in state_space:
    print(
        f"U^gamma_O({state} | pi_tr) = {berechne_nutzen_for_state(trials_for_state[state]):0.3f}"
    )

Aufgabe 1 - bestimmung von O_s
s_a^v: [[('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^w', 8), ('s_r^w', 'move', 's_a^w', 0)], [('s_a^v', 'load', 's_a^v', -1), ('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^v', 8), ('s_r^v', 'move', 's_a^v', 0), ('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^w', 8), ('s_r^w', 'move', 's_a^w', 0)], [('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^v', 8), ('s_r^v', 'move', 's_a^v', 0), ('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^w', 8), ('s_r^w', 'move', 's_a^w', 0)], [('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^w', 8), ('s_r^w', 'move', 's_a^w', 0)], [('s_a^v', 'load', 's_a^u', 3), ('s_a^u', 'move', 's_r^u', -1), ('s_r^u', 'unload', 's_r^u', -1), ('s_r^u', 'unload', 's_r^u', -1), ('s_r^u', 'unload', 's_r^w'

In [98]:
def conditional_probabilty(start_state, action, next_state):
    observations_with_same_start_and_action = [
        observation
        for observation in all_observations
        if observation[0] == start_state and observation[1] == action
    ]

    if len(observations_with_same_start_and_action) == 0:
        return 0

    observations_with_same_next_state_also = [
        observation
        for observation in observations_with_same_start_and_action
        if observation[2] == next_state
    ]

    return len(observations_with_same_next_state_also) / len(
        observations_with_same_start_and_action
    )


s_1 = "s_r^u"
s_2 = "s_a^v"

print("Für s_1:")
for next_state in state_space:
    print(
        f"\\tilde(P)(s_1, pi_tr(s), {next_state}) = {conditional_probabilty(s_1, strategy[s_1], next_state)}"
    )

print()
print("Für s_2:")
for next_state in state_space:
    print(
        f"\\tilde(P)(s_2, pi_tr(s), {next_state}) = {conditional_probabilty(s_2, strategy[s_2], next_state)}"
    )

Für s_1:
\tilde(P)(s_1, pi_tr(s), s_a^v) = 0.0
\tilde(P)(s_1, pi_tr(s), s_a^u) = 0.0
\tilde(P)(s_1, pi_tr(s), s_r^v) = 0.2
\tilde(P)(s_1, pi_tr(s), s_r^u) = 0.3
\tilde(P)(s_1, pi_tr(s), s_r^w) = 0.5
\tilde(P)(s_1, pi_tr(s), s_a^w) = 0.0

Für s_2:
\tilde(P)(s_2, pi_tr(s), s_a^v) = 0.125
\tilde(P)(s_2, pi_tr(s), s_a^u) = 0.875
\tilde(P)(s_2, pi_tr(s), s_r^v) = 0.0
\tilde(P)(s_2, pi_tr(s), s_r^u) = 0.0
\tilde(P)(s_2, pi_tr(s), s_r^w) = 0.0
\tilde(P)(s_2, pi_tr(s), s_a^w) = 0.0


In [103]:
zustandsnutzen_je_state = {state: 0 for state in state_space}


gamma = 0.9
for n, observation in enumerate(O_3, start=1):
    alpha = 1 / n
    first_state, action, next_state, reward = observation

    aktueller_zustandsnutzen = zustandsnutzen_je_state[first_state]
    beobachteter_zustandsnutzen = reward + gamma * zustandsnutzen_je_state[next_state]

    aktualisierter_zustandsnutzen = aktueller_zustandsnutzen + alpha * (
        beobachteter_zustandsnutzen - aktueller_zustandsnutzen
    )

    print(f"u^gamma({first_state} | pi_tr) = {aktualisierter_zustandsnutzen}")
    zustandsnutzen_je_state[first_state] = aktualisierter_zustandsnutzen

u^gamma(s_a^v | pi_tr) = 3.0
u^gamma(s_a^u | pi_tr) = -0.5
u^gamma(s_r^u | pi_tr) = -0.3333333333333333
u^gamma(s_r^u | pi_tr) = -0.575
u^gamma(s_r^u | pi_tr) = 1.14
u^gamma(s_r^w | pi_tr) = 0.0
